<center>
    <h1> ILI286 - Computación Científica II  </h1>
    <h2> EDP Hiperbólicas: Diferencias Finitas </h2>
    <h2> [S]cientific [C]omputing [T]eam<sup>1</sup></h2>
</center>

  
<br>
<br>
[1] _Material creado por profesor Claudio Torres_ (`ctorres@inf.utfsm.cl`) _y ayudantes: Alvaro Salinas y Martín Villanueva. DI UTFSM. Noviembre 2016._

# Tabla de Contenidos
* [Ecuación de Onda 1D](#onda)
* [Finite Differences con condición de borde de Dirichlet](#dirichlet)

### Librerías necesarias

In [12]:
%matplotlib inline
import numpy as np
import scipy.linalg as spla
import matplotlib.pyplot as plt
from ipywidgets import widgets
from ipywidgets import interact
import scipy.optimize

<div id='onda' />
# Ecuación de Onda 1D #

Consideremos la siguiente EDP hiperbólica, para $ x \in [a,b] \ \ $ y $t \in [0,T_{max}] \ \ $:
\begin{align*}
u_{tt}(x,t) &= c^2\, u_{xx}(x,t) \\
u(x,0) &= f(x)\\
u_t(x,0) &= g(x)\\
u(a,t) &= l(t)\\
u(b,t) &= r(t)
\end{align*}

Utilizando diferencias finitas, la ecuación nos queda:
\begin{align*}
u_{tt}(x,t) &= c^2\, u_{xx}(x,t) \\
\dfrac{u_i^{t+1}-2u_i^t+u_i^{t-1}}{\Delta t^2} &= c^2\, \dfrac{u_{i+1}^t-2u_i^t+u_{i-1}^t}{\Delta x^2} 
\end{align*}

Nuestra incógnita sería $u_i^{t+1}$, la cual se puede calcular teniendo información de los dos tiempos anteriores. Por lo tanto, despejando llegamos a:

$$u_i^{t+1} = \dfrac{c^2 \Delta t^2}{\Delta x^2}\left(u_{i+1}^t + u_{i-1}^t\right) + 2\left(1 - \dfrac{c^2 \Delta t^2}{\Delta x^2}\right)u_i^t - u_i^{t-1}$$

Si bien esto se resuelve directamente a medida que avanzamos en el tiempo, en $t=1$ nos faltará el valor de $u_i^{-1}$, pero recordando que:
$$u_t = \dfrac{u_i^{t+1} - u_i^{t-1}}{2\Delta t}$$
se puede utilizar la condición incial para obtener:
\begin{align*}
u_i^{t+1} &= \dfrac{c^2 \Delta t^2}{2 \Delta x^2}\left(u_{i+1}^t + u_{i-1}^t\right) + \left(1 - \dfrac{c^2 \Delta t^2}{\Delta x^2}\right)u_i^t + \Delta t u_t \\
u_i^1 &= \dfrac{c^2 \Delta t^2}{2 \Delta x^2}\left(u_{i+1}^0 + u_{i-1}^0\right) + \left(1 - \dfrac{c^2 \Delta t^2}{\Delta x^2}\right)u_i^0  + \Delta t u_t(x_i,0)
\end{align*}

<div id='dirichlet' />
## Finite Differences con condición de borde de Dirichlet ##

In [14]:
def Dirichlet(x,f,g,l,r,c2,dx,dt,N,M):
    
    sigma2 = c2 * dt**2 / dx**2
    
    print('sigma**2= ',sigma2)
        
    # The approximation for all time-steps
    w  = np.zeros((M+1,N))
    
    # Initial condition
    w[0,:]=f
        
    # Building A
    d=np.zeros(N-2)
    d[0]=2-2*sigma2
    d[1]=sigma2
    A=spla.toeplitz(d)
    
    ## Fist step
    tmp=np.zeros(N-2)
    tmp[0]=w[0,0]
    tmp[-1]=w[0,-1]
    w[1,1:-1]=0.5*(np.dot(A,w[0,1:-1]))+dt*g[1:-1]+0.5*sigma2*tmp
    
    ## All the rest of the steps
    for t in range(2,M+1):
        tmp[0]=l(dt*t)
        tmp[-1]=r(dt*t)
        w[t,1:-1]=np.dot(A,w[t-1,1:-1])-w[t-2,1:-1]+sigma2*tmp
    return w

In [15]:
def l(t):
    return 0*t

def r(t):
    return 0*t

In [16]:
def plot(us,i=0):
    plt.plot(x,us[i])
    plt.ylim(-0.11,0.11)
    plt.title('Tiempo: '+str(i*dt))
    plt.show()

In [17]:
N=100
M=200
TMAX = 40
x = np.linspace(-20,20,N)
t = np.linspace(0,TMAX,M)

c = 2

dx = x[1]-x[0]
dt = t[1]-t[0]

# RECALL: CFL condition:
#  if c dt/dx<=1 => stability
#  else (c dt/dx>1) instability. 

In [18]:
## Numerical Initial Conditions, notice these are numerical arrays!
# i.e. vectors not functions.
# u(x,0)=f(x)
f1 = 0.1*np.exp(-x**2)
f2 = 0.1*np.sin(2*x*np.pi/8)

# u_t(x,0)=f(x)
# SOLITON
#g1 = (np.exp(-(x-0.2)**2) - np.exp(-(x+0.2)**2))/2
g1 = np.zeros(N)
# ONDA QUE SE DIVIDE
g2 = np.zeros(N)

In [19]:
us1 = Dirichlet(x,f1,g1,l,r,c**2,dx,dt,N,M)

sigma**2=  0.989975000631


In [20]:
dp = 1
widgets.interact(plot,us=widgets.fixed(us1),i=(0,M,dp))

/Users/Roberto/anaconda/lib/python3.5/site-packages/traitlets/traitlets.py:567: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  silent = bool(old_value == new_value)


<function __main__.plot>

In [10]:
#Definimos las ecuaciones diferenciales planteadas en el enunciado
def F(y,a,b):
    return np.array([-b*y[0]*y[1],b*y[0]*y[1] - a*y[1],a*y[1]])
    
#resolvemos el problema con el metodo de euler, pasandole un initial guess y2 y los valores de alpha y beta.
def solve_SIR(y2,a,b):
    xi = [763,y2,0]
    #discretizamos el espacio en 200 puntos de 0 a 14, ya que conocemos los valores en ese extremo.
    time_grid = np.linspace(0,14,200)
    for i in range(len(time_grid)-1):
        #definimos un h pequeño, el cual sera los espacios entre punto y punto de nuestra discretización.
        h = time_grid[i+1] - time_grid[i]
        #actualizamos el vector con el metodo de euler.
        xi = xi + h*F(xi,a,b)
    #retornamos el valor de I(t) en 14.    
    return xi[1]

In [11]:
#Probamos nuestro algoritmo
initial_guess = 10

solx = solve_SIR(initial_guess,0.4477,0.0022)
print("Valor de I(14) sin aplicar busqueda de 0:")
print(solx)
print("\n")

#Definimos la funcion a la cual le aplicaremos la busqueda de 0 con broyden.
def g(x):
    return(solve_SIR(x,0.4477,0.0022) - 23)

x1 = scipy.optimize.broyden2(g,initial_guess)
print("Valor de I(14) aplicando busqueda de 0:")
print(x1)

Valor de I(14) sin aplicar busqueda de 0:
10.6190505186




NameError: name 'scipy' is not defined

In [21]:
def diferent_a_graphic(a):    
    
    initial_guess = 1

    solx = solve_SIR(initial_guess,a,0.0022)
    
    def g3(x):
        return(solve_SIR(x,a,0.0022) - 21)

    x1 = scipy.optimize.broyden2(g3,initial_guess)
    x = []
    y_1 = []
    y_2 = []
    y_3 = []
    xi = np.array([763,x1,0])
    eje_x = 0
    time_grid = np.linspace(0,14,200)
    for i in range(len(time_grid)-1):
        h = time_grid[i+1] - time_grid[i]
        eje_x = eje_x + h
        x.append(eje_x)
        y_1.append(xi[0])
        y_2.append(xi[1])
        y_3.append(xi[2])
        xi = xi + h*F(xi,a,0.0022)
    return x,y_1,y_2,y_3

In [24]:
def plot_a(a):
    x,y_1,y_2,y_3 = diferent_a_graphic(a)
    fig = plt.figure(figsize=(14, 7)) 
    plt.plot(x, y_1,'-y' ,label = 'suceptible')
    plt.plot(x, y_2,'-b', label = 'infectado')
    plt.plot(x, y_3,'-g', label = 'resistente')
    plt.grid(True)
    plt.grid(color = '0.5', linestyle = '--', linewidth = 1)
    plt.title("Avance de la poblacion con a = " + str(a),fontsize = 14, verticalalignment = 'baseline', horizontalalignment = 'center')
    plt.axis('tight')
    plt.legend(loc = 2,prop={'size':14})
    plt.xlabel("tiempo", fontsize = 14)
    plt.ylabel("población", fontsize = 14)
    plt.show()

In [25]:
dp = 0.03
widgets.interact(plot_a, a = (0.4477,0.8888,dp))

<function __main__.plot_a>